# Protocol for a strong coin flip

## Problem: 
Alice and Bob are on the telephone and they decide to flip a coin just for fun. But Alice can't see Bobs coin and viceversa because they are over the telephone. For the sake of the problem let's assume that both of them have access to a quantum computer and none of them want to cheat. As many other quantum cryptography algorithms it is necessary to make this kind of assumptions in order to get a useful result and we will see by the end of the post why this is necessary. So the proccess they have to follow is:

First let's define some imports.

In [1]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from random import randint
import hashlib

In the next cells Bob and Alice make an exchange of random bits using the bit commitment protocol. If you don't know the bit exchange protocol you can look it up [here](https://mentesniker.github.io/quantum-computer-science-tutorials/cryptography/BB84/).

In [77]:
#Alice cell, Bob can't see what's going in on here
m0 = randint(0,1)
Alice_base = randint(0,1)
qubits = list()
mycircuit = QuantumCircuit(1,1)
if(Alice_base == 0):
    if(m0 == 1):
        mycircuit.x(0)
else:
    if(m0 == 0):
        mycircuit.h(0)
    else:
        mycircuit.x(0)
        mycircuit.h(0)
qubits.append(mycircuit)

In [78]:
#Bob cell, Alice can't see what's going in on here
backend = Aer.get_backend('qasm_simulator')
a = 0
qubit = qubits[0]
if(Alice_base == 0):
    qubit.measure(0,0)
else:
    qubit.h(0)
    qubit.measure(0,0)
result = execute(qubit, backend, shots=1, memory=True).result()
a = int(result.get_memory()[0])

In [79]:
#Bob cell, Alice can't see what's going in on here
m1 = randint(0,1)
Bob_base = randint(0,1)
qubitsB = list()
mycircuitB = QuantumCircuit(1,1)
if(Bob_base == 0):
    if(m1 == 1):
        mycircuitB.x(0)
else:
    if(m1 == 0):
        mycircuitB.h(0)
    else:
        mycircuitB.x(0)
        mycircuitB.h(0)
qubitsB.append(mycircuitB)

In [80]:
#Alice cell, Bob can't see what's going in on here
backend = Aer.get_backend('qasm_simulator')
b = 0
qubit = qubitsB[0]
if(Bob_base == 0):
    qubit.measure(0,0)
else:
    qubit.h(0)
    qubit.measure(0,0)
result = execute(qubit, backend, shots=1, memory=True).result()
b = int(result.get_memory()[0])

Finally in order to obtain their shared coin flip they calculate the XOR between their two bits. In order to obtain the same bit the must do this operation because XOR with same operands give the same number.

In [81]:
#Alice cell, Bob can't see what's going in on here
print("The result of the coin flip was: " + str(b ^ m0))

The result of the coin flip was: 0


In [82]:
#Bob cell, Alice can't see what's going in on here
print("The result of the coin flip was: " + str(m1 ^ a))

The result of the coin flip was: 0


As you can see this protocol is just an extension of the original BB84 protocol. You may say that is an application of the protocol. But what happens when one or the two of them wants to cheat in the protocol?

## Why does it work?

Suppose that one of them want to cheat, for example Bob wants to obtain a 0 all the times. One of the places where they could cheat is when they apply the XOR. But since Alice played by the rules, Alice bit will be random and the resul of the protocol will be random. So Bob can't force Alice Bit to be wherever He chooses because Alice will immediately notice the change. Another time where Bob could have choosen to cheat was the bit agreament protocol. In order to cheat He would choose to lie to Alice about their shared key (a, b, m0 and m1 bits). However Alice will notice because both of them have the same information so their must be the same.

Everything is great except for the fact that both of them could choose to cheat. In that case the protocol fails because Alice would obtain a different bit compaired to Bob's bits and viceversa so none of them would obtain a truly random number.


# References:
Blum M.: Coin flipping by telephone a protocol for solving impossible problems. ACM SIGACT News
15(1), 23–27 (1983). doi:10.1145/1008908.1008911.
Broadbent, A., Schaffner, C. Quantum cryptography beyond quantum key distribution. Des. Codes Cryptogr. 78, 351–382 (2016). https://doi.org/10.1007/s10623-015-0157-4